In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [27]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/content/partial 3idt.pdf')

In [28]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [29]:
raw_text

'THE ORIGINAL  SCREENPLAYTHE ORIGINAL  SCREENPLAYEXT. AIRPORT – DAY\nA plane taxies on the runway.\nINT. AIRPLANE – DAY\nAn AIR HOSTESS closes the overhead \nbins. FARHAN (30s), a passenger, is reading a book when his phone rings. He scrambles for it, embarrassed he hadn’t turned it off. Other passengers look at him disapprovingly. He sheepishly takes the call.\nQjgku \ngSYyks gk¡A\nA beat.\nQjgku\n(incredulous)  \nD;k\\\nAIR HOSTESS\nSir, kindly switch off your mobile phone.\nFARHAN\nJust one sec, please, one sec, please.\nThe airplane revs its engines. Farhan hangs \nup, looking disturbed. He tries to catch the attention of the Air hostess.\nFARHAN \nExcuse me.\nThe plane races down the runway. \nFarhan appears to be in acute physical \ndiscomfort as the plane takes off. FARHAN \nHello... Yes?\nFARHAN \nWhat? 24FARHAN \n(to the Air hostess)Excuse me.\nHe unbuckles his seat belt and stands up. \nThe Air hostess is alarmed.\nAIR HOSTESS\nSir, please sit down.\nFarhan tries to steady hi

In [30]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [31]:
len(texts)

139

In [32]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [33]:
document_search = FAISS.from_texts(texts, embeddings)

In [34]:
document_search

In [35]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [36]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [45]:
query = "Enlist the lead characters involved in this movie"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\n1. Rancho\n2. Farhan\n3. Raju\n4. Pia\n5. Chatur\n6. Director'

In [43]:
query = "explain how the film starts in short?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

" The film starts at an airport where a passenger, Farhan, receives a call on his phone and is embarrassed that he forgot to turn it off. He tries to quickly answer the call before the plane takes off. The scene then shifts to a hostel room where one of the main characters, Joy, is seen playing guitar while Rancho observes him from a distance. The next scene shows the trio entering a room, looking like they just woke up. The Director then holds up an egg from a bird's nest and drops it, declaring that the competition is over and their life begins with murder. The film then shows Raju's house, a small and dingy room with a paralysed father, a coughing mother, and an unwed sister. Raju's mother complains about their financial difficulties and the pressure to get Raju's sister married."

In [42]:
query = "how many setting can the whole film be divided in?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

" The whole film can be divided into at least four settings: the Director's office at ICE, the corridor adjoining the courtyard where Millimeter irons clothes, Farhan and Raju's families' homes, and Farhan's room. However, there may be additional settings throughout the film."